In [ ]:
import pandas as pd
import numpy as np
import urllib
import random
import datetime
import re

In [ ]:
# restaurants dataset
df_restaurants = pd.read_csv("C:\\Users\\hsahn\\Design of AI systems\\module 7\\restaurants.csv")
df_weather = pd.read_csv("C:\\Users\\hsahn\\Design of AI systems\\module 7\\weather.csv") 
df_bus = pd.read_csv("C:\\Users\\hsahn\\Design of AI systems\\module 7\\bus.csv") 
df_restaurants = df_restaurants.sort_values(by=['Cuisine', 'Rating'], ascending=[True, False])

#df_weather
#df_bus
#df_restaurants

In [ ]:
# pd.set_option('display.max_rows', None)
# df_weather

In [ ]:
restaurant_list_from_df = df_restaurants.values.tolist()
weather_list_from_df = df_weather.values.tolist()
bus_list_from_df = df_bus.values.tolist()

In [ ]:
class Restaurant:
    def __init__(self):
        self.form = [['cuisine',''],['dietary options','']]
        self.cuisine_options = ['chinese', 'french', 'greek', 'indian', 'italian', 'japenese', 'mexican', 'thai']
        self.dietary_options = ['meat', 'vegetarian', 'vegan']
        self.cuisine_pattern = re.compile(r'\b(' + '|'.join(self.cuisine_options) + r')\b')
        self.dietary_pattern = re.compile(r'\b(' + '|'.join(self.dietary_options) + r')\b')

    def fill_form(self, user_input):
        
        # Extract cuisine
        cuisine_match = self.cuisine_pattern.search(user_input.lower())
        if cuisine_match:
            self.form[0][1] = cuisine_match.group(1).capitalize() # update value for cuisine

        # Extract dietary options
        dietary_match = self.dietary_pattern.search(user_input.lower())
        if dietary_match:
            self.form[1][1] = dietary_match.group(1).capitalize() # update value for dietary_options
                
    def check_form(self):

        # Check if cuisine is filled, else ask relevant questions
        if not self.form[0][1]:
            print("I understood you would like to go to a restaurant, can you specify the cuisine?")
            input_string = input("What is your preference? ").lower()
            cuisine_match = self.cuisine_pattern.search(input_string)
            if cuisine_match:
                self.form[0][1] = cuisine_match.group(1).capitalize()
            else:
                while self.form[0][1] == '':
                    print("I did not quite understand, I can help you with the following options")
                    print(', '.join(self.cuisine_options))
                    input_string = input("What is your preference? ").lower()
                    cuisine_match = self.cuisine_pattern.search(input_string)
                    if cuisine_match:
                        self.form[0][1] = cuisine_match.group(1).capitalize()
        
        # Check if dietary options is filled, else ask relevant questions
        if self.form[1][1] == '':
            input_string = input("I understood you would like to find a restaurant which serves " + self.form[0][1] + " cuisine. Could you tell about your dietary options? ").lower()
            dietary_match = self.dietary_pattern.search(input_string)
            if dietary_match:
                self.form[1][1] = dietary_match.group(1).capitalize()
            else:
                while self.form[1][1] == '':
                    print("I did not quite understand, Would you prefer Vegetarian, Vegan or Meat")
                    input_string = input("What is your preference? ").lower()
                    dietary_match = self.dietary_pattern.search(input_string)
                    if dietary_match:
                        self.form[1][1] = dietary_match.group(1).capitalize()
                        
        self.recommend_restaurants(df_restaurants)
        
    def recommend_restaurants(self, df):
        if self.form[0][1] == '' or self.form[1][1] == '': # handle as no preference -> return all
            print("I need more information before I can recommend a restaurant.")
            return

        
        result = df.loc[(df['Cuisine'] == self.form[0][1]) & (df['Dietary options'] == self.form[1][1].capitalize())]
        result = result[['Name','Rating','Opening Hours']].sort_values('Rating', ascending=False)
        if result.size == 0:
            print(f"Sorry, there exist no restaurant that serves {self.form[1][1]} {self.form[0][1]} dishes.")
        else:
            print(f"The following restaurant(s) serve(s) {self.form[1][1]} {self.form[0][1]} dishes:")
            print(result.to_string(index=False))


In [ ]:
# restaurant = Restaurant()
# user_input = "I would like to eat food."
# restaurant.fill_form(user_input)
# restaurant.check_form()

In [ ]:
class Weather:
    def __init__(self):
        self.form = [['city',''],['date','']]
        self.cities = ['stockholm', 'gothenburg', 'malmo', 'uppsala', 'lund', 'umea', 'boras', 'halmstad']
        self.days = [str(i) for i in range(1, 32)]
        self.months = [str(i) for i in range(1, 13)]
        self.city_pattern = re.compile(r'\b(' + '|'.join(self.cities) + r')\b')
        self.date_pattern = re.compile(r'(\d{1,2})-(\d{1,2})(-\d{2,4})?$')# DD-MM or DD-MM-YY or DD-MM-YYYY
        
    # if possible, fill form with user's first input
    def fill_form(self, user_input):
     
        # Extract city
        city_match = self.city_pattern.search(user_input.lower()) # check match in whole string
        if city_match:
            self.form[0][1] = city_match.group(1).capitalize()

        # Extract date
        self.match_date(user_input)
     
    # however, in case the user did not provide enough info.. inquire!
    def check_form(self):
        if not self.form[0][1]:
            print("I understand you would like to know the weather.")
            while not self.form[0][1]:
                city = input("Can you specify the city? ").lower()
                if city == 'exit':
                    break
                if city in self.cities:
                    self.form[0][1] = city.capitalize()
                else:
                    print("I did not quite understand. Here are the cities I can help you with:")
                    print(", ".join(self.cities))
        
        if not self.form[1][1]:
            print(f"I understand you would like to know the weather for {self.form[0][1]}.")
            while not self.form[1][1]:
                date = input("What date?")
                if date == 'exit':
                    break
                res = self.match_date(date)
                if res:
                    break
#                 else:
#                     print("I did not quite understand.")
        
        # now that the form has been filled, collect the data and present to the user
        self.report_weather(df_weather)

    def report_weather(self, df):
        result = df.loc[(df['City'] == self.form[0][1]) & (df['Date'] == self.form[1][1])]
        result = result[['Time','Weather','Temperature']]
        if result.size == 0:
            print(f"Sorry, there is no weather data available for {self.form[0][1]} on {self.form[1][1]}.")
        else:
            print(f"The weather report for {self.form[0][1]} on {self.form[1][1]} is as follows :")
            print(result.to_string(index=False))
            
    def match_date(self, input):
        date_match = self.date_pattern.search(input.lower())
        if date_match:
            date = ""
            day = date_match.group(1)
            month = date_match.group(2)
            year = date_match.group(3)
            if year is None:
                date = f"{day}-{month}-2023"
            elif len(year) == 2:
                date = f"{day}-{month}-20{year}"
            elif year is None and month is not None and day is not None:
                date = date_match.group()
            else:    
                print("Invalid date.")
                return True # invalid
            self.form[1][1] = date
            return False # valid

In [ ]:
# weather = Weather()
# user_input = "I want to know the weather"
# weather.fill_form(user_input)
# weather.check_form()

In [ ]:
class Bus:
    def __init__(self):
        self.form = [['from','',''],['to','','']]
        self.stops =['brunnsparken', 'saltholmen', 'heden', 'lindholmen', 'kallebäck']
        self.from_stop_regex = re.compile(r'(?<=from\s)(?:{})'.format('|'.join(self.stops)))
        self.to_stop_regex = re.compile(r'(?<=to\s)(?:{})'.format('|'.join(self.stops)))
        self.from_time_regex = re.compile(r'(?<=leave\sat\s)(\d{1,2}:\d{2})')
        self.to_time_regex = re.compile(r'(?<=arrive\sat\s)(\d{1,2}:\d{2})')

    # Extract stops
    def fill_form(self, user_input):
        from_stop_match = self.from_stop_regex.search(user_input.lower())
        to_stop_match = self.to_stop_regex.search(user_input.lower())
        from_time_match = self.from_time_regex.search(user_input.lower())
        to_time_match = self.to_time_regex.search(user_input.lower())
                                          
        if from_stop_match:
            self.form[0][1] = from_stop_match.group().capitalize()
        if from_time_match:
            self.form[0][2] = from_time_match.group()
        if to_stop_match:
            self.form[1][1] = to_stop_match.group().capitalize()
        if to_time_match:
            self.form[1][2] = to_time_match.group()

    # Check if form is filled, otherwise, ask for more information
    def check_form(self):
        # Check if "from" is filled
        if self.form[0][1] == '':   
            word = input("I understood you would like to travel, but from where?").lower()
            if word in self.stops:
                self.form[0][1] = word.capitalize()
            else:
                while self.form[0][1] == '':
                    word = input("I did not quite understand, could you specify the location that you want to travel from?").lower()
                    if word in self.stops:
                        self.form[0][1] = word.capitalize()
        
        # Check if "to" is filled
        if self.form[1][1] == '':   
            word = input("I understood you would like to travel from " + self.form[0][1] + " but to where?").lower()
            if word in self.stops:
                self.form[1][1] = word.capitalize()
            else:
                while self.form[1][1] == '':
                    word = input("I did not quite understand, could you specify the destination?")
                    if word in self.stops:
                        self.form[1][1] = word.capitalize()
                        
        self.plan_trip(df_bus)
        
    def plan_trip(self, df):
        result = df.loc[(df['From'] == self.form[0][1]) & (df['To'] == self.form[1][1])].sort_values('Departure')
        if result.size == 0:
            print(f"Sorry, there are no available connections leaving from {self.form[0][1]} and going to {self.form[1][1]}.")
        else:
            answer = f"You can take the following buses to go from {self.form[0][1]} to {self.form[1][1]}"
            if self.form[0][2] != '':
                result = result[result['Departure'] < self.form[0][2]]
                answer += f", with departure time {self.form[0][2]} at latest :"
            elif self.form[1][2] != '':
                result = result[result['Arrival'] < self.form[1][2]]
                answer += f", with arrival time no later than {self.form[1][2]} :"
            print(answer)
            print(result.to_string(index=False))

In [ ]:
# bus = Bus()
# # user_input = "I want to go from Brunnsparken to Heden. I'd like to arrive at 12:30"
# user_input = "travel"
# bus.fill_form(user_input)
# bus.check_form()

In [ ]:
#Method to extract context from intitial question
def extract_context(sentence):
    weather_pattern = re.compile(r"(weather|temperature)") # key_words = ['weather','sunny','rain','cloudy','windy','hot','cold','temperature']
    bus_pattern = re.compile(r"(bus|travel|go)") # key_words = ['bus','tram','bus stop','travel','train', 'transport']
    restaurant_pattern = re.compile(r"(restaurant|food|eat)") # key_words = ['restaurant','restaurants','food','hungry', 'dish', 'sushi', 'hamburger', 'pizza','cuisine']
    
    context = ''
    if weather_pattern.search(sentence):
        context = Weather()
    elif bus_pattern.search(sentence):
        context = Bus()
    elif restaurant_pattern.search(sentence):
        context = Restaurant()
    return context

In [ ]:
#Main chatbot method
def get_help():    
    print("Hello. I'm Bot. You can ask me about weather, restaurants and the bus schedule.")
    end = False
    
    while not end: 
        inputString = input("What can I help you with?").lower()
        if inputString == 'exit':
            break
        context = extract_context(inputString)
        context.fill_form(inputString)
        context.check_form()
    print("Goodbye. Have a nice day!")
    
get_help()